In [ ]:
import pandas as pd
import numpy as np
import os
import sys
import pickle

In [ ]:
# "53-0.0": first day of attending 
# 31-0.0: gender
# 34-0.0: year of birth
# 845-1.0	845-2.0: age completed full time education
# 6138: QUALIFICATION


In [ ]:
root_path = os.path.join(os.path.dirname(os.path.dirname(os.getcwd())))

with open(os.path.join(root_path, "data\\metadata\\target_col_idx.pickle"), 'rb') as f:
    target_col_idx = pickle.load(f)

target_col_idx.append("eid")

metadata = pd.read_csv(os.path.join(root_path, "data/metadata/metadata_subset.csv"), usecols=target_col_idx)
metadata.set_index("eid", inplace=True)
# data_coding = pd.read_csv(os.path.join(root_path, "data/metadata/Codings.csv"), 
#                           encoding='latin-1')
# data_dict = pd.read_csv(os.path.join(root_path, "data/metadata/Data_Dictionary_Showcase.csv"))

In [ ]:
disease_dict = { 
# FieldID: Coding
 "20002": [1081, 1086, 1491, 1583, 1261, 1262, 1263, 1397], 
 "41203": [430, 431, 434, 436, 340, 332, 290, 341], 
 "41205": [430, 431, 434, 436, 340, 332, 290, 341],
 "41202": ["I60", "I61", "I63", "I64", "G35", "G20", "F00", "F01", "F02", "F03", 
           "G30", "G31", "G32", "Block G30-G32", "G36", "G37"], 
 "41204": ["I60", "I61", "I63", "I64", "G35", "G20", "F00", "F01", "F02", "F03", 
           "G30", "G31", "G32", "Block G30-G32", "G36", "G37"], 
 }

In [ ]:
# selected_col = []
# all_col = list(metadata.columns)

# for field in disease_dict.keys():
#     # target_cols = [(idx, col) for idx, col in enumerate(all_col) if field in col]
#     target_cols = [col for col in (all_col) if field in col]
#     selected_col.extend(target_cols)

# with open(os.path.join(root_path, "data\\metadata\\target_col_idx.pickle"), 'wb') as f:
#     pickle.dump(selected_col, f)

In [ ]:
all_col = list(metadata.columns)
# selected_col = []
metadata_concate = pd.DataFrame(index=metadata.index)
metadata_concate["is_selected"] = True

for field in disease_dict.keys():
    target_cols = [col for col in all_col if field in col]
    # concatenate the cols under the same field
    metadata_concate[field] = metadata[target_cols].apply(
    lambda x: ','.join(x.dropna().astype(str)),
    axis=1)
    
    # concate all the target disease
    all_disease = "".join(str(i)+"|" for i in disease_dict[field])
    all_disease = all_disease[:-1]
    
    # the person is tagged as unhealthy if any of the target diseases exist.
    is_healthy = ~metadata_concate[field].str.contains(all_disease)
    metadata_concate["is_selected"] = metadata_concate["is_selected"] & is_healthy
metadata_concate
# metadata = metadata[selected_col]

In [ ]:
metadata_concate[metadata_concate["is_selected"] == False]